In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2"

import sys
sys.path.append('/home/rustam/hyperspecter_segmentation/makitorch')
sys.path.append('/home/rustam/hyperspecter_segmentation/')
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.ndimage import gaussian_filter
import cv2
from hsi_dataset_api import HsiDataset
from makitorch.dataloaders.HsiDataloader import HsiDataloader

import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms as T
import torchvision.transforms.functional as TF
from torchvision import utils
import cv2
from Losses import FocalLoss
import matplotlib.pyplot as plt

import seaborn as sns
import json

In [ ]:
PREFIX_INFO_PATH = '/home/rustam/hyperspecter_segmentation/danil_cave/kfolds_data/kfold0'
PATH_DATA = '/raid/rustam/hyperspectral_dataset/new_cropped_hsi_data'


pca_explained_variance = np.load(f'{PREFIX_INFO_PATH}/kfold0_PcaExplainedVariance_.npy')
pca_mean = np.load(f'{PREFIX_INFO_PATH}/kfold0_PcaMean.npy')
pca_components = np.load(f'{PREFIX_INFO_PATH}/kfold0_PcaComponents.npy')


test_indices = np.load(f'{PREFIX_INFO_PATH}/kfold0_indx_test.npy')
train_indices = np.load(f'{PREFIX_INFO_PATH}/kfold0_indx_train.npy')


In [ ]:
def pca_transformation(x):
    x_t = np.reshape(x, (x.shape[0], -1)) # (C, H, W) -> (C, H * W)
    x_t = np.swapaxes(x_t, 0, 1) # (C, H * W) -> (H * W, C)
    x_t = x_t - pca_mean
    x_t = np.dot(x_t, pca_components.T) / np.sqrt(pca_explained_variance)
    return np.reshape(x_t, (x.shape[1], x.shape[2], pca_components.shape[0])).astype(np.float32) # (H, W, N)

In [ ]:
def test_augmentation(image, mask):
    image = TF.to_tensor(image)
    #image = (image - image.min()) / (image.max() - image.min())
    
    mask = torch.from_numpy(mask)
    
    mask = torch.squeeze(mask, 0)
    return image, mask


def preprocessing(imgs, masks):
    with open(f'{PREFIX_INFO_PATH}/data_standartization_params_kfold0.json', 'r') as f:
        data_standartization_params = json.load(f)
    mean = data_standartization_params.get('means')
    std = data_standartization_params.get('stds')
    assert mean is not None and std is not None
    def standartization(img):
        return np.array((img - mean) / std, dtype=np.float32)
    _images = [np.transpose(image, (1, 2, 0)) for image in imgs] #[pca_transformation(image) for image in imgs]
    #_images = [standartization(image) for image in _images]
    _masks = [
        np.expand_dims(
            cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY).astype(np.uint8)
            ,0
        ).astype(np.int64)
        for mask in masks
    ]
    return _images, _masks

In [ ]:
dataset_test = HsiDataloader(
    PATH_DATA, preprocessing=preprocessing, 
    augmentation=test_augmentation, indices=test_indices
)

dataset_train = HsiDataloader(
    PATH_DATA, preprocessing=preprocessing,
    augmentation=test_augmentation, indices=train_indices
)

In [ ]:
val_loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=1, num_workers=10)

val_loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=1)

In [ ]:
from tqdm import tqdm

In [ ]:
val_loader = val_loader_train

specter_list = []
target_list = []

for img_s, mask_s in tqdm(val_loader):
    specter_list.append(img_s[0].numpy())
    target_list.append(mask_s[0].numpy())
len(specter_list)

In [ ]:
indx = 6
indx_sp = 0

In [ ]:
sns.heatmap(specter_list[indx][indx_sp])

In [ ]:
sns.heatmap(target_list[indx])

In [ ]:
cube = specter_list[indx][:, :32, :32].copy().reshape(-1)
sns.distplot(cube)

In [ ]:
cube = specter_list[indx][indx_sp].copy().reshape(-1)
sns.distplot(cube)

In [ ]:
plt.imshow((specter_list[indx][indx_sp] * 255).astype(np.uint8))

In [ ]:
import seaborn as sns
import pandas as pd

In [ ]:
num_class2count_test = dict([(str(i), 0) for i in range(17)])
num_class2count_train = dict([(str(i), 0) for i in range(17)])

num_class2count_pixels_test = dict([(str(i), 0) for i in range(17)])
num_class2count_pixels_train = dict([(str(i), 0) for i in range(17)])

# Test count
for img_s, mask_s in tqdm(val_loader_test):
    for n_c in np.unique(mask_s):
        num_class2count_test[str(n_c)] += 1

# Train count
for img_s, mask_s in tqdm(val_loader_train):
    for n_c in np.unique(mask_s):
        num_class2count_train[str(n_c)] += 1

# Test pixels
for img_s, mask_s in tqdm(val_loader_test):
    for n_c in np.unique(mask_s):
        num_class2count_pixels_test[str(n_c)] += torch.sum(mask_s == n_c).numpy()

# Train pixels
for img_s, mask_s in tqdm(val_loader_train):
    for n_c in np.unique(mask_s):
        num_class2count_pixels_train[str(n_c)] += torch.sum(mask_s == n_c).numpy()

In [ ]:
sns.barplot(data=pd.DataFrame(num_class2count_train, index=[0]) / 324)

In [ ]:
sns.barplot(data=pd.DataFrame(num_class2count_test, index=[0]) / 37)

In [ ]:
num_class2count_diff = dict()

for k,v in num_class2count_test.items():
    num_class2count_diff[str(k)] = v - num_class2count_train[k]
    print(f'class={str(k).zfill(2)} num_test={str(v).zfill(2)} num_train={str(num_class2count_train[k]).zfill(2)}')

In [ ]:
sns.barplot(data=pd.DataFrame(num_class2count_diff, index=[0]))

In [ ]:
sns.barplot(data=pd.DataFrame(num_class2count_pixels_train, index=[0]))

In [ ]:
sns.barplot(data=pd.DataFrame(num_class2count_pixels_test, index=[0]))

In [ ]:
num_class2count_pixels_diff = dict()

for k,v in num_class2count_pixels_test.items():
    num_class2count_pixels_diff[str(k)] = v - num_class2count_pixels_train[k]

In [ ]:
sns.barplot(data=pd.DataFrame(num_class2count_pixels_diff, index=[0]))